Author: Geoff Boeing <br />
Web: http://geoffboeing.com/2015/03/chaos-theory-logistic-map/  <br />
Date: 2015-03-30 <br />
Description: This notebook demonstrates how to make animated GIFs that pan and zoom around 3-D Poincaré plots.

In [1]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image as PIL_Image
from images2gif import writeGif
import glob
import IPython.display as IPdisplay
import matplotlib.cm as cm

## The first 3 cells just contain functions for the logistic map and creating the 3D plots

In [2]:
# for documentation of this function, see chaos-logistic-model.ipynb
def logistic_model(generations=20, growth_rate_min=0.5, growth_rate_max=4.0, growth_rate_steps=7, pop_initial=0.5):
    growth_rate_min = float(growth_rate_min)
    growth_rate_max = float(growth_rate_max) - 0.0000000001
    growth_rates = np.arange(growth_rate_min, growth_rate_max, (growth_rate_max - growth_rate_min) / growth_rate_steps)
    pops = pd.DataFrame(columns=growth_rates, index=range(generations))
    pops.iloc[0] = pop_initial
    for rate in pops.columns:
        pop = pops[rate]
        for t in range(generations - 1):
            pop[t + 1] = pop[t] * rate * (1 - pop[t])
    return pops

In [3]:
def get_poincare_points(pops, discard_gens=1, dimensions=2):
    """
    convert a dataframe of values from the logistic model into a set of xy(z) points to plot
    
    pops = population data output from the model
    discard_gens = number of rows to discard before keeping points to plot
    dimensions = 2 or 3, if we want points for a 2-D or 3-D plot
    """
    
    # drop the first row because every run has the same starting value, it leaves a visual artifact
    # drop the initial rows of the populations data, if specified by the argument
    if discard_gens > 0:
        discard_gens = np.arange(0, discard_gens)
        pops = pops.drop(labels=pops.index[discard_gens])
    
    # a point is defined by the name of its model run then its spatial coordinates
    points = []
    point_columns = ['name', 'x', 'y', 'z']
    
    # for each column in the logistic populations dataframe, where the label is the 'name' of the model run
    for name in pops.columns:
        
        # for each row in the column
        for label, row in pops.iterrows():
            
            # we can only create points up up to row dimensions-1 because we need future time steps to create each point
            if label < len(pops)-(dimensions-1):
                
                point = [name]
                for n in range(dimensions):
                    # append the value at the current time (aka row) as x, t+1 as y (and t+2 as z if dimensions=3)
                    point.append(pops[name][label + n])
                
                # append this point to the list of points
                points.append(point)
    
    # convert the list of points to a MultiIndex DataFrame 
    # with a level in the index called 'name' to represent each model run
    df = pd.DataFrame(points, columns=point_columns[0:dimensions+1])
    df.index = pd.MultiIndex.from_tuples(zip(df['name'], df.index), names=['name', ''])
    df = df.drop(labels='name', axis=1)
    return df

In [4]:
def get_poincare_plot_3d(pops, discard_gens=1, height=8, width=10, 
                     xmin=0, xmax=1, ymin=0, ymax=1, zmin=0, zmax=1,
                     title='', elev=25, azim=240, dist=10, depthshade=True,
                     xlabel='Population (t)', ylabel='Population (t + 1)', zlabel='Population (t + 2)',
                     marker='.', size=5, alpha=0.7, color='r', color_list=[], legend=False,
                     legend_bbox_to_anchor=None, remove_ticks=True):
    
    # first get the xyz points to plot
    points = get_poincare_points(pops, discard_gens, dimensions=3)
    plots = []
    
    # get_poincare_points() returns a MultiIndexed DataFrame, each run of the model has its own 'name' in the index
    index = points.index.get_level_values('name')
    names = np.unique(index)
    
    # add a color to the list for each run of the model if list doesn't have the same length as the number of runs
    # that way, each model run scatterplot gets a color, and we default to 'r' if color_list isn't passed or long enough
    color_list = [color for n in range(len(names))] if len(color_list) < len(names) else color_list
    
    # create a new figure, set its size, and create an axis with a 3-D projection
    fig = plt.figure(figsize=(width, height))
    ax = fig.gca(projection='3d')

    # configure the perspective from which to view the 3D plot
    ax.elev = elev
    ax.azim = azim
    ax.dist = dist
    
    # set the plot title, axis limits, and axis labels
    ax.set_title(title)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_zlim(zmin, zmax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)  
    ax.set_zlabel(zlabel)
    
    # remove all ticks is argument is True
    if remove_ticks:
        ax.tick_params(reset=True, axis='both', which='both', pad=0, width=0, length=0,
                       bottom='off', top='off', left='off', right='off', 
                       labelbottom='off', labeltop='off', labelleft='off', labelright='off')
    else:
        ax.tick_params(reset=True)
    
    # plot the xyz data for each run of the model that appears in the MultiIndex
    for n in range(len(names)):
        xyz = points.iloc[index == names[n]]
        plots.append(ax.scatter(xyz['x'], xyz['y'], xyz['z'], depthshade=depthshade,
                                marker=marker, c=color_list[n], edgecolor=color_list[n], s=size, alpha=alpha))
        
    # add a legend if argument is True
    if legend:
        ax.legend(plots, names.tolist(), loc=1, frameon=True, framealpha=1, bbox_to_anchor=legend_bbox_to_anchor)
        
    return fig, ax

##Create a 3-D poincare plot as an animated gif that pans, rotates, and zooms. This demonstrates how the viewing perspective is composed of an elevation, a distance, and an azimuth.

In [5]:
# set a filename, run the logistic model, and create the plot
gif_filename = 'demo-pan-rotate-zoom'
pops = logistic_model(1000, 3.99, 4, 1)
fig, ax = get_poincare_plot_3d(pops, remove_ticks=False)

# create 36 frames for the animated gif
steps = 36

# a viewing perspective is composed of an elevation, distance, and azimuth
# define the range of values we'll cycle through for the distance of the viewing perspective
min_dist = 7.
max_dist = 10.
dist_range = np.arange(min_dist, max_dist, (max_dist-min_dist)/steps)

# define the range of values we'll cycle through for the elevation of the viewing perspective
min_elev = 10.
max_elev = 60.
elev_range = np.arange(max_elev, min_elev, (min_elev-max_elev)/steps)

# now create the individual frames that will be combined later into the animation
for azimuth in range(0, 360, 360/steps):
    
    # pan down, rotate around, and zoom out
    ax.azim = float(azimuth)
    ax.elev = elev_range[int(azimuth/(360./steps))]
    ax.dist = dist_range[int(azimuth/(360./steps))]
    
    # set the figure title to the viewing perspective, and save each figure as a .png
    fig.suptitle('elev=' + str(round(ax.elev,1)) + ', azim=' + str(round(ax.azim,1)) + ', dist=' + str(round(ax.dist,1)))
    plt.savefig('images/' + gif_filename + '/img' + str(azimuth).zfill(3) + '.png')
    
# don't display the static plot...
plt.close()

# ...instead, create an animated gif of all the frames, then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.2)
IPdisplay.Image(url=file_path_name)

##Create a 3-D poincare plot as an animated gif starts by looking straight down at the x-y plane (this is what a 2-D plot would look like), then panning and rotating around to show the 3-D structure

In [6]:
# set a filename, run the logistic model, and create the plot
gif_filename = 'logistic-3d-return-map'
pops = logistic_model(1000, 3.99, 4, 1)
fig, ax = get_poincare_plot_3d(pops, color='m', xlabel='Population (t)', ylabel='Population (t + 1)', zlabel='')

# look straight down at the x-y plane to start off
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 11.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n > 19 and n < 23:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 22 and n < 37:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 36 and n < 61:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n > 60 and n < 65:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n > 64 and n < 74:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n > 73 and n < 77:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position
    
    if n > 76: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Population (t)')
        ax.set_ylabel('Population (t + 1)')
        ax.set_zlabel('Population (t + 2)')
        
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'3D Poincaré Plot, r=3.99', fontsize=12, x=0.5, y=0.85)
    plt.savefig('images/' + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

##Do the same thing again, but this time plot both the chaotic logistic model output and random noise

In [7]:
# run the logistic model and create random noise
chaos_pops = logistic_model(1000, 3.99, 4, 1)
random_pops = pd.DataFrame([random.random() for _ in range(0, 1000)], columns=['value'])
pops = pd.concat([chaos_pops, random_pops], axis=1)
pops.columns = ['chaos', 'random']

In [8]:
# set a filename and then create the plot
gif_filename = 'chaos-vs-random-return-map'
fig, ax = get_poincare_plot_3d(pops, color_list=['m', 'c'], xlabel='Population (t)', ylabel='Population (t + 1)', zlabel='', 
                               legend=True, legend_bbox_to_anchor=(0.94, 0.96))

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 11.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n > 19 and n < 23:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 22 and n < 37:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 36 and n < 61:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n > 60 and n < 65:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n > 64 and n < 74:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n > 73 and n < 77:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position
    
    if n > 76: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Population (t)')
        ax.set_ylabel('Population (t + 1)')
        ax.set_zlabel('Population (t + 2)')
        
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'3-D Poincaré Plot, chaos vs random', fontsize=12, x=0.5, y=0.85)
    plt.savefig('images/' + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

## Create a 3-D Poincaré Plot to show the logistic map's strange attractors across the chaotic regime (from r=3.6 to r=4.0), twisting and curling around their state space in three dimensions. Animated it by panning and rotating to reveal the structure and its odd folds.

In [9]:
# run the model for 2,000 generations for 50 growth rate parameters between 3.6 and 4.0
steps = 50
pops = logistic_model(2000, 3.6, 4.0, steps)
points = get_poincare_points(pops, discard_gens=1, dimensions=3)

# get the color map and turn it into a list of colors, one for each model run
num_of_colors = steps
colors = cm.get_cmap('hsv')([x/float(num_of_colors) for x in range(num_of_colors)]).tolist()
colors.reverse()

In [10]:
# set a filename and create the plot
gif_filename = 'multiple-chaos-return-map'
fig, ax = get_poincare_plot_3d(pops, color_list=colors, xlabel='Population (t)', ylabel='Population (t + 1)', zlabel='')

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 11.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n > 19 and n < 23:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 22 and n < 37:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 36 and n < 61:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n > 60 and n < 65:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n > 64 and n < 74:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n > 73 and n < 77:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position
    
    if n > 76: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Population (t)')
        ax.set_ylabel('Population (t + 1)')
        ax.set_zlabel('Population (t + 2)')
    
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'3-D Poincaré Plot, r=3.6 to r=4.0', fontsize=12, x=0.5, y=0.85)
    plt.savefig('images/' + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/' + gif_filename + '/*.png')]
file_path_name = 'images/' + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)